The aim of this exercise is to train you in debugging networks using the good old print function and also tensorboard. To simulate poor training, we will train a multilayer perceptron using the CIFAR data.

1. Use the CIFAR data set reader from the first homework and read the CIFAR-10 files again. 
2. Apply random noise to the image 
3. Convert the image to float and scale to [0.0, 1.0] by dividing the pixel values by the highest pixel value.
4. Convert all labels to onehot encoding
5. Build a 3-layer multilayer perceptron of size [512, 256, 128]. 
6. Create a tensorboard summary for plotting the histogram of the weights of the three layers.
7. Also write the cost / loss at the end of each epoch to tensorboard.
8. Train the network with learning rates of [0.1, 0.01, 0.001]. You will notice that the network will not converge well.
9. Submit the snapshot of the histograms for the three learning rates. Describe your observations.



In [1]:
import pickle
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import pandas as pd 
import random
import datetime

random.seed(3457)

#1. Use the CIFAR data set reader from the first homework and read the CIFAR-10 files again. 
files = glob.glob(
    "/Users/omerorhan/Desktop/UCSC/Deep Learning/cifar-10-batches-py/data_batch*")

def unpickle(file, types):
    with open(file, 'rb') as fo:
        return pickle.load(fo, encoding='bytes').get(types)

data = np.concatenate([unpickle(x, b'data') for x in files])
labels = np.concatenate([unpickle(x, b'labels') for x in files])




/anaconda3/envs/tf-cpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#2. Apply random noise to the image 
mean = 1   # some constant
std = 1    # some constant (standard deviation)
noisy_data = data + np.random.normal(mean, std, data.shape)

#3. Convert the image to float and scale to [0.0, 1.0] by dividing the pixel values by the highest pixel value.
noisy_data = noisy_data.astype(float)
noisy_data = np.divide(noisy_data, np.amax(noisy_data[0]))

noisy_data = np.float32(noisy_data)


In [4]:
#4. Convert all labels to onehot encoding
n_values = np.max(labels) + 1
print(type(labels))
labelsencoded = np.eye(n_values)[labels]
labelsencoded = np.float32(labelsencoded)

print(labelsencoded[25])

<class 'numpy.ndarray'>
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [5]:
# Hyper-parameters

learningrate = 0.001
nepochs = 100
batch_size = 128
#128,3072
nhidden1 = 512
nhidden2 = 256 
nhidden3 = 128 
ninput = noisy_data[0].flatten().shape[0]
noutput = 10

X = tf.placeholder(tf.float32, [batch_size,ninput])
Y = tf.placeholder(tf.float32, [batch_size, noutput])


print(X)
print(Y)

Tensor("Placeholder:0", shape=(128, 3072), dtype=float32)
Tensor("Placeholder_1:0", shape=(128, 10), dtype=float32)


In [6]:
#5. Build a 3-layer multilayer perceptron of size [512, 256, 128]. 

weights = \
{
        'h1': tf.Variable(tf.random_normal([ninput, nhidden1])),
        'h2': tf.Variable(tf.random_normal([nhidden1, nhidden2])),
        'h3': tf.Variable(tf.random_normal([nhidden2, nhidden3])),
        'out': tf.Variable(tf.random_normal([nhidden3, noutput]))
}

biases = \
{
    'b1': tf.Variable(tf.random_normal([nhidden1])),
    'b2': tf.Variable(tf.random_normal([nhidden2])),
    'b3': tf.Variable(tf.random_normal([nhidden3])),
    'out': tf.Variable(tf.random_normal([noutput]))
}

def multiperceptron(x):
    l1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    l2 = tf.nn.sigmoid(tf.add(tf.matmul(l1, weights['h2']), biases['b2']))
    l3 = tf.nn.sigmoid(tf.add(tf.matmul(l2, weights['h3']), biases['b3']))
    outl = tf.nn.sigmoid(tf.add(tf.matmul(l3, weights['out']), biases['out']))
    return outl

model = multiperceptron(X)


Instructions for updating:
Colocations handled automatically by placer.


In [7]:

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningrate)
train_min = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
# argmax tells what index is the max in the y hat
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("accuracy", accuracy)

#6. Create a tensorboard summary for plotting the histogram of the weights of the three layers.
# Add to tensorboard summary
tf.summary.histogram("weight_1",weights['h1'])
tf.summary.histogram("weight_2",weights['h2'])
tf.summary.histogram("weight_3",weights['h3'])
tf.summary.scalar("loss", loss)

merged_summary_op = tf.summary.merge_all()
filename="/Users/omerorhan/Desktop/UCSC/Deep Learning/DeepLearningUCSC/venv/homework/Assignments" \
         "/logs/run"+datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S_learningrate"+str(learningrate))
writer = tf.summary.FileWriter(filename, tf.get_default_graph())
init = tf.global_variables_initializer()


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
import random
def getbatch(alldata, alllabels, batch_size = 16):
    nlabels = len(alllabels)
    number_of_batches = nlabels//batch_size # TODO: Change 100 to nlabels
    for batch_number in range(number_of_batches):
        rand_index = [random.randrange(0, nlabels) for i in range(batch_size)]
        batch_x = alldata[rand_index]
        batch_y = np.array([alllabels[idx] for idx in rand_index])
        yield (batch_x, batch_y)
 


In [9]:

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(nepochs):
        for batch_x, batch_y in getbatch(noisy_data, labelsencoded, batch_size):
            sess.run(train_min, feed_dict={X:batch_x, Y:batch_y})
        losscalc, accuracycalc, merged_summary = \
                      sess.run([loss, accuracy, merged_summary_op], feed_dict={X:batch_x, Y:batch_y})
        writer.add_summary(merged_summary, epoch) 
        
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f"%(epoch, losscalc, accuracycalc))
writer.close()

Epoch: 0, Loss: 2.1841, Accuracy: 0.0938


Epoch: 1, Loss: 2.0179, Accuracy: 0.3203


Epoch: 2, Loss: 2.0523, Accuracy: 0.2891


Epoch: 3, Loss: 2.0568, Accuracy: 0.3281


Epoch: 4, Loss: 2.0231, Accuracy: 0.3906


...

Epoch: 95, Loss: 1.8742, Accuracy: 0.4141


Epoch: 96, Loss: 1.8659, Accuracy: 0.4453


Epoch: 97, Loss: 1.9262, Accuracy: 0.4062


Epoch: 98, Loss: 1.8161, Accuracy: 0.5156


Epoch: 99, Loss: 1.8920, Accuracy: 0.4688


### Accuracy(learning rate 0.1):
In the figure below, you will notice that the accuracy is around 10%. 
Due to learning rate(0.1), accuracy is very bad.

<img src="hm2_img/lr0.1accuracy.png" height="200px">


### Loss(learning rate 0.1):
In the figure below, you will notice that the loss is around 2.2 - 2.38%. 

<img src="hm2_img/lr0.1loss.png" height="200px">

### Accuracy(learning rate 0.01):
In the figure below, you will notice that the accuracy is around 10%. 
Due to learning rate(0.01), accuracy is very bad.

<img src="hm2_img/lr0.01accuracy.png" height="200px">


### Loss(learning rate 0.01):
In the figure below, you will notice that the loss is around 2.2 - 2.38%. 

<img src="hm2_img/lr0.01loss.png" height="200px">


### Accuracy(learning rate 0.001):
In the figure below, you will notice that the accuracy is around 10%. 
Due to learning rate(0.001), accuracy is very bad.

<img src="hm2_img/lr0.001accuracy.png" height="200px">


### Loss(learning rate 0.001):
In the figure below, you will notice that the loss is around 2.2 - 2.38%. 

<img src="hm2_img/lr0.001loss.png" height="200px">
